In [46]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV


from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [47]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.sample(3)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
1190,22/12/2020 17:11:31,37.0,Male,Aachen,QA Engineer,15,1,Senior,Python,NaN,70000.0,7000,28,Full-time employee,Unlimited contract,English,101-1000,Product
235,24/11/2020 13:55:19,32.0,Male,Berlin,Engineering Manager,9,5,Lead,NaN,"PHP, Javascript / Typescript, .NET, SQL, Googl...",95000.0,140000,365,Full-time employee,Unlimited contract,English,1000+,Product
1209,28/12/2020 22:24:47,24.0,Female,Munich,Software Engineer,less than year,less than year,student,"Python, database technologies","Python, Java / Scala, SQL, AWS",16320.0,16320,10,Part-time employee,Unlimited contract,English,11-50,Startup


drop timestamp column first, and make functions for missing value counts and replacement of NaN values.

In [48]:
df.drop(columns = {'Timestamp'}, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 17 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Age                                                     1226 non-null   float64
 1   Gender                                                  1243 non-null   object 
 2   City                                                    1253 non-null   object 
 3   Position                                                1247 non-null   object 
 4   Total years of experience                               1237 non-null   object 
 5   Years of experience in Germany                          1221 non-null   object 
 6   Seniority level                                         1241 non-null   object 
 7   Your main technology / programming language             1126 non-null   object 
 8   Other technologies/programming languag

In [49]:
def replace_low_freq(d, col, threshold=10, replacement='other'):
    value_counts = d[col].value_counts() # Specific column 
    to_remove = value_counts[value_counts <= threshold].index
    tmp = d[col].replace(to_replace=to_remove, value=replacement)
    return tmp

In [50]:
def missing_values() :
    for c in df.columns :
        count = df[c].isnull().sum()
        print(f"missing values in column '{c}': {count}")

In [51]:
def commas(df, columns) :
    df[columns] = pd.to_numeric(df[columns], downcast = 'float', errors = 'coerce')
    df[columns].replace(',', '.', inplace = True)
    df[columns].apply(lambda n : f'{n : ,.1f}')
    print(df[columns].sample(5))

In [52]:
# Construct Target by adding salary and bonus. 
df["Yearly bonus + stocks in EUR"] = pd.to_numeric(df["Yearly bonus + stocks in EUR"], downcast="float", errors="coerce")
df["Yearly bonus + stocks in EUR"].fillna(0, inplace=True)
df['total salary'] = df["Yearly brutto salary (without bonus and stocks) in EUR"] + df["Yearly bonus + stocks in EUR"]
df.drop(columns={"Yearly brutto salary (without bonus and stocks) in EUR", "Yearly bonus + stocks in EUR"}, inplace=True)
df.sample(3)

,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type,total salary
814,35.0,Female,Berlin,Product Manager,13,6,Senior,NaN,NaN,26,Full-time employee,Unlimited contract,English,101-1000,Product,85000.0
635,33.0,Male,Berlin,DevOps,8,3,Senior,AWS,"Python, AWS, Kubernetes, Docker",30,Full-time employee,Unlimited contract,English,1000+,service,76000.0
681,32.0,Male,Berlin,Backend Developer,10,3,Lead,Ruby,"Python, Javascript / Typescript, Ruby, SQL, AW...",28,Full-time employee,Unlimited contract,English,51-100,Product,80000.0


<b> missing values check: </b>

1. check missing values in the Age column, and compare them with other columns. if the missing values also appear with other columns, it is better to drop those 27 rows.
2. check the total salary plot to check outliers. slicing the value up to 250000.00.
3. plot the gender column, and change diverse to others. replace the 3 missing values with 'Other'.
4. check the Seniority level column and sort them to others that count under 10.
5. 'Total years of experience' and 'Years of experience in Germany' columns had error symbols for floating numbers. They were converted from ',' to '.'. Then changed the type to float64 and replaced by NaN values to 0.
6. 'Main language at work' column sorting into 5 language categories. 
7. 'Company types' column sorting into 4 categories and sorting them to others that count under 10.
8. grouping by value count that under 10 types of company types into others and replace NaN values to an unknown type.
9. converted 0 to the rest of the columns that have missing values. 
10. drop 'Number of vacation days', 'Employment status', 'Сontract duration' columns, they are not very helpful for the prediction.
11. combined 'Your main technology / programming language' and 'Other technologies/programming languages you use often' columns into 1 column of programming language. Replace NaN values to an unknown type.


In [53]:
missing_values()

missing values in column 'Age': 27
missing values in column 'Gender': 10
missing values in column 'City': 0
missing values in column 'Position ': 6
missing values in column 'Total years of experience': 16
missing values in column 'Years of experience in Germany': 32
missing values in column 'Seniority level': 12
missing values in column 'Your main technology / programming language': 127
missing values in column 'Other technologies/programming languages you use often': 157
missing values in column 'Number of vacation days': 68
missing values in column 'Employment status': 17
missing values in column 'Сontract duration': 29
missing values in column 'Main language at work': 16
missing values in column 'Company size': 18
missing values in column 'Company type': 25
missing values in column 'total salary': 0


In [54]:
df[df['Age'].isnull()][['Total years of experience', 'Seniority level', 'Employment status', 'Company type']]

,Total years of experience,Seniority level,Employment status,Company type
11,25,Senior,Self-employed (freelancer),Product
12,NaN,Lead,Full-time employee,NaN
28,14,Senior,Full-time employee,Product
55,NaN,Senior,Full-time employee,NaN
113,6,Middle,Full-time employee,Product
300,7.5,Middle,Full-time employee,Product
330,2,Middle,Full-time employee,Product
340,NaN,Senior,Full-time employee,NaN
365,9,Lead,Full-time employee,Startup
374,8,Lead,Part-time employee,Product


In [55]:
df.dropna(subset=['Age'], inplace = True)
df.sample(3)

,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type,total salary
1203,33.0,Female,Munich,Software Engineer,10,3,Senior,Java,SQL,25,Full-time employee,Unlimited contract,English,11-50,Outsourse,47500.0
420,31.0,Female,Munich,Software Engineer,2,2,Middle,Java,SQL,28,Full-time employee,Unlimited contract,German,1000+,Product,61000.0
62,34.0,Male,Munich,Frontend Developer,5,5,Senior,JavaScript/ES6,NaN,27,Full-time employee,Unlimited contract,English,11-50,Product,73500.0


In [56]:
fig = px.box(df, y = 'total salary', title = 'Total salary + bonus + stock in EUR Box Plot')
fig.show()

In [57]:
out1 = df.loc[(df['total salary'] > 250000.00)]
out1

,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type,total salary
279,39.0,Male,Berlin,Product Manager,10,5,Lead,NaN,NaN,25,Full-time employee,Unlimited contract,English,1000+,Product,3.200000e+05
441,32.0,Male,Berlin,CTO,10,4,CTO,JAVA,"Python, C/C++",28,Full-time employee,Unlimited contract,English,101-1000,Product,4.000000e+05
494,39.0,Male,Berlin,Manager,11,9,Head,Python,"Python, Kotlin, Java / Scala, AWS",28,Full-time employee,Unlimited contract,English,1000+,Product,3.300000e+05
564,30.0,Male,Berlin,Data Engineer,5,4,Lead,NaN,Python,26,Full-time employee,Unlimited contract,English,1000+,Product,1.240000e+06
574,38.0,Male,Berlin,Freelance AI lead / advisor,20,7,Head,C,"Python, C/C++, Docker",0,Self-employed (freelancer),0,English,up to 10,Consulting / Agency,4.000000e+05
576,24.0,Male,Munich,Data Scientist,1,1,Junior,C++,"Python, C/C++, Swift, Java / Scala, R, SQL, Go...",NaN,Full-time employee,NaN,English,101-1000,Startup,3.000000e+05
630,40.0,Male,Cupertino,Software Engineer,20,0,Lead,Swift,NaN,45,Full-time employee,Unlimited contract,English,1000+,Product,7.000000e+05
732,36.0,Male,Munich,DevOps,16,0,Senior,SRE,"Python, C/C++, Go",30,Full-time employee,Unlimited contract,English,1000+,Product,2.550000e+05
745,35.0,Male,Berlin,Software Engineer,13,5,Lead,TypeScript,"Python, Javascript / Typescript, SQL, AWS, Goo...",29,Full-time employee,Unlimited contract,English,101-1000,Startup,3.030000e+05
778,42.0,Male,Berlin,Frontend Developer,18,5,Senior,React JS,Javascript / Typescript,35,Full-time employee,Unlimited contract,English,101-1000,Product,9.450000e+05


In [58]:
to_drop1 = out1.index[:14].tolist()
df = df.drop(to_drop1)
df.sample(3)

,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type,total salary
460,32.0,Male,Hamburg,Software Engineer,12,7,Senior,Java,"Kotlin, PHP, Javascript / Typescript, Java / S...",28,Full-time employee,Unlimited contract,German,101-1000,Product,67300.0
112,33.0,Male,Berlin,Backend Developer,12,6,Lead,PHP,"Javascript / Typescript, SQL, AWS, Docker",27,Full-time employee,Unlimited contract,English,101-1000,Product,65000.0
199,32.0,Male,Munich,Fullstack Developer,9,2,Senior,Java,"Javascript / Typescript, Kubernetes, Docker, A...",26,Full-time employee,Unlimited contract,English,11-50,Product,74000.0


In [59]:
df['total salary'].describe()

count      1212.000000
mean      81757.482327
std       35807.218216
min       10001.000000
25%       60000.000000
50%       73000.000000
75%       90000.000000
max      240000.000000
Name: total salary, dtype: float64

In [60]:
fig = px.box(df, y = 'total salary', title= 'Total salary + bonus + stock in EUR Box Plot')
fig.show()

In [61]:
df['Seniority level'].value_counts()

Senior                                       550
Middle                                       358
Lead                                         158
Junior                                        77
Head                                          41
Principal                                      3
intern                                         1
student                                        1
Intern                                         1
Entry level                                    1
Working Student                                1
C-Level                                        1
no idea, there are no ranges in the firm       1
Student                                        1
Key                                            1
C-level executive manager                      1
Director                                       1
No level                                       1
Work Center Manager                            1
Manager                                        1
VP                  

In [62]:
df['Seniority level'].replace(np.nan, 'other', inplace = True)
df['Seniority level'] = replace_low_freq(df, 'Seniority level', 10, 'Other')
df['Seniority level'].value_counts()

Senior    550
Middle    358
Lead      158
Junior     77
Head       41
Other      28
Name: Seniority level, dtype: int64

In [63]:
commas(df, 'Total years of experience')
commas(df, 'Years of experience in Germany')

1126     4.0
1218     8.0
627      4.0
1065     4.0
354     15.0
Name: Total years of experience, dtype: float32
583    5.0
923    3.0
469    3.0
435    4.0
490    6.0
Name: Years of experience in Germany, dtype: float32


In [64]:
df['Total years of experience'].replace('less than year', '0', inplace = True)
df['Total years of experience'].fillna(0, inplace = True)
df['Years of experience in Germany'].replace(np.nan, 0, inplace = True)
df.loc[df['Years of experience in Germany'] < 1.0, 'Years of experience in Germany'] = 0

In [65]:
df['Main language at work'].value_counts()

English               988
German                181
Russian                12
Italian                 3
Spanish                 3
English and German      2
Polish                  2
Русский                 2
Czech                   2
50/50                   1
Deuglisch               1
French                  1
both                    1
Russian, English        1
Name: Main language at work, dtype: int64

In [66]:
df['Main language at work'].fillna('Unknown', inplace = True)
df['Main language at work'] = replace_low_freq(df, 'Main language at work', 10, 'Other')
df['Main language at work'].value_counts()

English    988
German     181
Other       19
Unknown     12
Russian     12
Name: Main language at work, dtype: int64

In [67]:
df['Company type'] = replace_low_freq(df, 'Company type', 10, 'other')
df['Company type'].value_counts()

Product                739
Startup                246
Consulting / Agency    138
other                   73
Name: Company type, dtype: int64

In [68]:
df['City'] = replace_low_freq(df, 'City', 5, 'other')
df['City'].value_counts()

Berlin        656
Munich        230
other         161
Frankfurt      42
Hamburg        38
Stuttgart      26
Cologne        19
Amsterdam       9
Stuttgart       7
Düsseldorf      6
Moscow          6
Karlsruhe       6
Prague          6
Name: City, dtype: int64

In [69]:
df.rename(columns = {'Position ' : 'Position'}, inplace = True)
df['Gender'].fillna('Other', inplace = True)
df['Position'].fillna('unknown', inplace = True)
df['Total years of experience'].fillna(0, inplace = True)
df['Company size'].fillna('Unknown', inplace = True)
df['Company type'].fillna('Unknown', inplace = True)

df['Your main technology / programming language'].fillna('Unknown', inplace = True)
df['Other technologies/programming languages you use often'].fillna('Unknown', inplace = True)

replace_low_freq(df, 'Company type', 10, 'other')
df.drop(columns = {'Number of vacation days', 'Employment status', 'Сontract duration'}, inplace = True)


In [70]:
df['Your main technology / programming language'].fillna('Unknown', inplace = True)
df['Your main technology / programming language'].value_counts()

Java                 179
Python               161
Unknown              117
PHP                   54
C++                   35
                    ... 
SAP BW / ABAP          1
Go, PHP, SQL           1
Kotlin                 1
Jira                   1
consumer analysis      1
Name: Your main technology / programming language, Length: 253, dtype: int64

In [71]:
df['Other technologies/programming languages you use often'].fillna('Unknown', inplace = True)
df['Other technologies/programming languages you use often'].value_counts()

Unknown                                                           146
Javascript / Typescript                                            42
Python                                                             36
SQL                                                                31
AWS, Docker                                                        15
                                                                 ... 
Javascript / Typescript, Java / Scala, AWS, Kubernetes, Docker      1
Kotlin, Java / Scala, SQL, Kubernetes, Docker                       1
Python, C/C++, Javascript / Typescript, SQL, Docker                 1
Google Cloud, Kubernetes, Docker, Shell                             1
Python, AWS, Kubernetes, Docker, terraform, ansible                 1
Name: Other technologies/programming languages you use often, Length: 552, dtype: int64

In [72]:
df['programming language'] = df['Your main technology / programming language'] + ',' + df['Other technologies/programming languages you use often']
df.drop(columns={'Your main technology / programming language', 'Other technologies/programming languages you use often'}, inplace = True)
df.sample(5)

,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Main language at work,Company size,Company type,total salary,programming language
1129,33.0,Male,Munich,Backend Developer,13.0,3.0,Senior,English,51-100,Product,77000.0,"Java,Python, Javascript / Typescript, AWS"
224,35.0,Male,Berlin,Software Engineer,15.0,4.0,Senior,English,51-100,Startup,70000.0,"go,PHP, Javascript / Typescript, Perl, AWS, Ku..."
816,33.0,Female,Berlin,Data Scientist,8.0,4.0,Senior,English,1000+,Product,77000.0,"Python ,SQL, Google Cloud, Docker"
490,29.0,Male,Munich,Software Engineer,8.0,6.0,Middle,English,1000+,Product,75000.0,"PHP,Javascript / Typescript, SQL"
978,30.0,Male,Frankfurt,Data Scientist,5.0,4.0,Middle,English,11-50,Startup,52500.0,"Python,Javascript / Typescript, Java / Scala, ..."


In [73]:
missing_values()

missing values in column 'Age': 0
missing values in column 'Gender': 0
missing values in column 'City': 0
missing values in column 'Position': 0
missing values in column 'Total years of experience': 0
missing values in column 'Years of experience in Germany': 0
missing values in column 'Seniority level': 0
missing values in column 'Main language at work': 0
missing values in column 'Company size': 0
missing values in column 'Company type': 0
missing values in column 'total salary': 0
missing values in column 'programming language': 0


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1212 entries, 0 to 1252
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Age                             1212 non-null   float64
 1   Gender                          1212 non-null   object 
 2   City                            1212 non-null   object 
 3   Position                        1212 non-null   object 
 4   Total years of experience       1212 non-null   float32
 5   Years of experience in Germany  1212 non-null   float32
 6   Seniority level                 1212 non-null   object 
 7   Main language at work           1212 non-null   object 
 8   Company size                    1212 non-null   object 
 9   Company type                    1212 non-null   object 
 10  total salary                    1212 non-null   float64
 11  programming language            1212 non-null   object 
dtypes: float32(2), float64(2), object(

In [75]:
df['Total years of experience'] = df['Total years of experience'].apply(lambda x: np.float64(x))
df['Years of experience in Germany'] = df['Years of experience in Germany'].apply(lambda x: np.float64(x))

#df[['Gender', 'City', 'Position', 'Seniority level', 'Main language at work', 'Company size',
#       'Company type', 'programming language']] = df[['Gender', 'City', 'Position', 'Seniority level',
#                                                      'Main language at work', 'Company size', 'Company type', 'programming language']].astype('category')

In [76]:
df.select_dtypes(include = np.number).columns

Index(['Age', 'Total years of experience', 'Years of experience in Germany',
       'total salary'],
      dtype='object')

In [77]:
df.select_dtypes(exclude = np.number).columns

Index(['Gender', 'City', 'Position', 'Seniority level',
       'Main language at work', 'Company size', 'Company type',
       'programming language'],
      dtype='object')

1. built numeric_features, categorical_features, numeric_transformer, categorical_transformer to pipeline.

2. since cleaned all the missing and null values, the scaler can chosen StandardScaler.

3. for categorical data, this data set used ExtraTreeRegressor and RandomForestRegressor.

4. defined data set for training and testing data sets.

In [78]:
numeric_features = ['Age', 'Total years of experience', 'Years of experience in Germany',
       'total salary']
numeric_transformer = Pipeline( steps = [
        ("imputer", SimpleImputer()),
        ("scaler", StandardScaler())])

categorical_features = ['Gender', 'City', 'Position', 'Seniority level',
                         'Main language at work', 'Company size', 'Company type']
categorical_transformer = Pipeline( steps = [
        ("imputer", SimpleImputer(strategy = 'most_frequent')),
        ("encode", OneHotEncoder())])


preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [79]:
dfm = df
dfm = pd.get_dummies(dfm, drop_first = True)
y = np.array(dfm['total salary']).reshape(-1,1)
x = np.array(dfm.drop(columns = {'total salary'}))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

<b> Pipeline 1 WITHOUT feature selection

In [80]:
#Build pipeline
pipe1 = [('pre', preprocessor), ('reg', ExtraTreeRegressor(random_state= 0))]
pipeline1 = Pipeline(pipe1)
pipeline1 = Pipeline([('clf', ExtraTreeRegressor())])


pipeline1.fit(x_train, y_train)
y_pred1 = pipeline1.predict(x_test)
cv1 = cross_val_score(ExtraTreeRegressor(), x, y, cv = 5)

print('Training set accuracy: {:.2f}'.format(pipeline1.score(x_train, y_train)))
print('Testing set accuracy: {:.2f}'.format(pipeline1.score(x_test, y_test)))
print('RMSE: {:.2f}'.format(np.sqrt(mean_squared_error(y_test, y_pred1))))
print('R2: {:.2f}'.format(r2_score(y_test, y_pred1)))
print('Cross Validation score: {:.2f}'.format(cv1.mean()))

Training set accuracy: 1.00
Testing set accuracy: -0.04
RMSE: 37274.91
R2: -0.04
Cross Validation score: -0.03


<b> Pipeline 2 WITHOUT feature selection

In [81]:
dfm = pd.get_dummies(dfm, drop_first = True)
y = np.array(dfm['total salary']).reshape(-1)
x = np.array(dfm.drop(columns = {'total salary'}))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

# Create the pipeline
pipe2 = Pipeline([('pre', preprocessor), ('reg', RandomForestRegressor(n_estimators = 10, random_state = 0))])
pipeline2 = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators = 5, random_state=0))


# Fit the pipeline to the training data
pipeline2.fit(x_train, y_train)
y_pred2 = pipeline2.predict(x_test)
cv2 = cross_val_score(RandomForestRegressor(), x, y, cv = 5)


# Evaluate the pipeline on the test data
print('Training set accuracy: {:.2f}'.format(pipeline2.score(x_train, y_train)))
print('Testing set accuracy: {:.2f}'.format(pipeline2.score(x_test, y_test)))
print('RMSE: {:.2f}'.format(np.sqrt(mean_squared_error(y_test, y_pred2))))
print('R2: {:.2f}'.format(r2_score(y_test, y_pred2)))
print('Cross Validation score: {:.2f}'.format(cv2.mean()))

Training set accuracy: 0.81
Testing set accuracy: 0.12
RMSE: 34257.57
R2: 0.12
Cross Validation score: 0.16


<b> Feature selection with VarianceThreshold

In [82]:
var_th = VarianceThreshold(.2)
y = dfm['total salary']
x = dfm.drop('total salary', axis = 1)

post_vt = var_th.fit_transform(x)
print(x.shape)
print(post_vt.shape)

mask = var_th.get_support()
new_features = x.columns[mask]
print(new_features)

(1212, 1118)
(1212, 9)
Index(['Age', 'Total years of experience', 'Years of experience in Germany',
       'City_Berlin', 'Position_Software Engineer', 'Seniority level_Middle',
       'Seniority level_Senior', 'Company size_101-1000',
       'Company type_Product'],
      dtype='object')


<b> Feature selection with mutual_info_regression

In [83]:
y = dfm['total salary']
x = dfm.drop('total salary', axis = 1)
mi = mutual_info_regression(x, y)

mi_scores = list(zip(x.columns, mi))
mi_scores = sorted(mi_scores, key=lambda x: x[1], reverse=True)


# print the mi_scores list
print('Mutual information scores:')
for i, s in mi_scores:
    print('{} : {:.2f}'.format(i, s))

Mutual information scores:
Total years of experience : 0.18
Years of experience in Germany : 0.12
Seniority level_Middle : 0.12
Seniority level_Senior : 0.08
Age : 0.08
City_other : 0.06
programming language_JS,Javascript / Typescript, Google Cloud, Kubernetes, Docker : 0.06
programming language_Python,R, SQL, AWS, Azure, Docker : 0.05
programming language_Java,Kotlin, SQL, Kubernetes, Docker : 0.04
programming language_C#,Javascript / Typescript, .NET, Azure, Docker : 0.04
programming language_Java,Python, Javascript / Typescript, SQL, AWS, Kubernetes : 0.04
programming language_Python ,Python, Docker : 0.04
Position_Head of IT  : 0.03
Position_unknown : 0.03
Position_Backend Developer : 0.03
programming language_PHP,PHP, Javascript / Typescript, Java / Scala : 0.03
programming language_Java,Java / Scala, AWS, Azure, Kubernetes, Docker : 0.03
programming language_Kubernetes,Python, AWS, Google Cloud, Kubernetes, Docker : 0.03
Position_Frontend Developer : 0.03
programming language_Sca

<b> Pipeline 3, 4 with feature selection from above

In [84]:
fsy = dfm['total salary']
fsx = x[new_features]
fsx_train, fsx_test, fsy_train, fsy_test = train_test_split(fsx, fsy, test_size = 0.3, random_state = 0)

et_pipeline = make_pipeline(VarianceThreshold(.2), StandardScaler(), ExtraTreesRegressor())
rf_pipeline = make_pipeline(VarianceThreshold(.2), StandardScaler(), RandomForestRegressor())


# Fit the pipelines on the new data
et_pipeline.fit(fsx, fsy)
rf_pipeline.fit(fsx, fsy)

Pipeline(steps=[('variancethreshold', VarianceThreshold(threshold=0.2)),
                ('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor())])

In [85]:
# ExtraTreeRegressor
fsy_pred3 = et_pipeline.predict(fsx_test)
cv3 = cross_val_score(ExtraTreeRegressor(), fsx, fsy, cv = 5)

print('Training set accuracy: {:.2f}'.format(et_pipeline.score(fsx_train, fsy_train)))
print('Testing set accuracy: {:.2f}'.format(et_pipeline.score(fsx_test, fsy_test)))
print('RMSE: {:.2f}'.format(np.sqrt(mean_squared_error(fsy_test, fsy_pred3))))
print('R2: {:.2f}'.format(r2_score(fsy_test, fsy_pred3)))
print('Cross Validation score: {:.2f}'.format(cv3.mean()))

Training set accuracy: 0.98
Testing set accuracy: 0.99
RMSE: 2913.17
R2: 0.99
Cross Validation score: -0.68


In [86]:
# RandomForestRegressor
fsy_pred4 = rf_pipeline.predict(fsx_test)
cv4 = cross_val_score(RandomForestRegressor(), fsx, fsy, cv = 5)

print('Training set accuracy: {:.2f}'.format(rf_pipeline.score(fsx_train, fsy_train)))
print('Testing set accuracy: {:.2f}'.format(rf_pipeline.score(fsx_test, fsy_test)))
print('RMSE: {:.2f}'.format(np.sqrt(mean_squared_error(fsy_test, fsy_pred4))))
print('R2: {:.2f}'.format(r2_score(fsy_test, fsy_pred4)))
print('Cross Validation score: {:.2f}'.format(cv4.mean()))

Training set accuracy: 0.86
Testing set accuracy: 0.87
RMSE: 13311.18
R2: 0.87
Cross Validation score: 0.02


<b> Define Hyperparameters and Grid Search CV to look better model. (Extra Tree regressor, Random Forest Regressor)

In [87]:
fsy = dfm['total salary']
fsx = x[new_features]
fsx_train, fsx_test, fsy_train, fsy_test = train_test_split(fsx, fsy, test_size = 0.3, random_state = 0)

In [88]:
param_grid1 = {
    'extratreesregressor__n_estimators': [50, 100, 200],
    'extratreesregressor__max_depth': [5, 10, 15],
    'extratreesregressor__min_samples_leaf': [1, 2, 3, 4, 5],
}

In [89]:
# param_grid1
et_pipeline = make_pipeline(VarianceThreshold(.2), StandardScaler(), ExtraTreesRegressor())

grid = GridSearchCV(et_pipeline, param_grid = param_grid1, cv = 5, n_jobs=-1) 
grid.fit(fsx_train, fsy_train)


fsy_pred = grid.best_estimator_.predict(fsx_test)
cv5 = cross_val_score(ExtraTreesRegressor(), fsx, fsy, cv = 5)
best = grid.best_estimator_

In [90]:
print('best hyperparameters:', grid.best_params_)
print('score: {:.2f}'. format(grid.best_score_))

print('\nTraining set accuracy: {:.2f}'.format(best.score(fsx_train, fsy_train)))
print('R2 score: {:.2f}'.format(best.score(fsx_test, fsy_test)))
print('RMSE: {:.2f}'.format(np.sqrt(mean_squared_error(fsy_test, fsy_pred))))
print('Cross Validation score: {:.2f}'.format(cv5.mean()))

best hyperparameters: {'extratreesregressor__max_depth': 5, 'extratreesregressor__min_samples_leaf': 4, 'extratreesregressor__n_estimators': 200}
score: 0.18

Training set accuracy: 0.26
R2 score: 0.12
RMSE: 34282.95
Cross Validation score: -0.07


In [91]:
param_grid2 = {
    'randomforestregressor__n_estimators': [50, 100, 200],
    'randomforestregressor__max_depth': [5, 10, 15],
    'randomforestregressor__min_samples_leaf': [1, 2, 3, 4, 5],
}

In [92]:
# param_grid2
rf_pipeline = make_pipeline(VarianceThreshold(.2), StandardScaler(), RandomForestRegressor())

grid = GridSearchCV(rf_pipeline, param_grid = param_grid2, cv = 5, n_jobs=-1) 
grid.fit(fsx_train, fsy_train)


fsy_pred = grid.best_estimator_.predict(fsx_test)
cv6 = cross_val_score(RandomForestRegressor(), fsx, fsy, cv = 5)
best = grid.best_estimator_

In [93]:
print('best hyperparameters:', grid.best_params_)
print('score: {:.2f}'. format(grid.best_score_))

print('\nTraining set accuracy: {:.2f}'.format(best.score(fsx_train, fsy_train)))
print('R2 score: {:.2f}'.format(best.score(fsx_test, fsy_test)))
print('RMSE: {:.2f}'.format(np.sqrt(mean_squared_error(fsy_test, fsy_pred))))
print('Cross Validation score: {:.2f}'.format(cv6.mean()))

best hyperparameters: {'randomforestregressor__max_depth': 5, 'randomforestregressor__min_samples_leaf': 5, 'randomforestregressor__n_estimators': 50}
score: 0.18

Training set accuracy: 0.32
R2 score: 0.17
RMSE: 33441.99
Cross Validation score: 0.02


# Answers and Explainations
- in the total salary column, there were some outliers after the combined salary, bonus and stocks columns.
- dropped these columns: 'Number of vacation days', 'Employment status', 'Сontract duration'
- checked the rest of the other columns and deleted a few rows for data wrangling. (1212 rows, 12 columns)
- for categorical data, models used Extra Tree regressor and Random Forest Regressor for all the setups.
- since all the missing values and null values were done, the scaler can be chosen StandardScaler.
- for categorical data, this data set used ExtraTreeRegressor and RandomForestRegressor
- for feature selection, VarianceThreshold and mutual_info_regression were used. Comparing both results, they had similar columns and a couple of different column orders.
- in hyperparameter changes, estimators were from 50 to 500, the result shows the best was between 100-200. The maxmium depth was not over 20, and the minimum sample leaf was not over 5.


### Results
- the results for both regressors were really bad without feature selection and hyperparameter changes. After feature selection, the most relevant features for the prediction can actually improve the model performance.
- As the R2 score increased to 0.99 and 0.87, there probably had overfitting issues during the process.
- used the same Hyperparameter settings for both regressors, the RandomForestRegressor had slightly better performance.
- the RandomForestRegressor model with Feature Selection and Hyperparameter Changes was the better model.



-- without feature selection and hyperparameters
- Extra Tree regressor

    training data accuracy <1.00>, RMSE <38013.45>, R2 <-0.08>, CV_score <0.00>

- Random Forest Regressor

    training data accuracy <0.81>, RMSE <34257.57>, R2 <0.12>, CV_score <0.16>

### Feature Selection Activities
-- with feature selection
- Extra Tree regressor

    training data accuracy <0.98>, RMSE <2913.17>, R2 <0.99>, CV_score <-0.68>

- Random Forest Regressor

    training data accuracy <0.86>, RMSE <13311.18>, R2 <0.87>, CV_score <0.02>

### Hyperparameter Changes
-- with feature selection and Hyperparameter Changes
- Extra Tree regressor

    training data accuracy <0.26>, RMSE <34282.85>, R2 <0.12>, CV_score <-0.07>

- Random Forest Regressor

    training data accuracy <0.32>, RMSE <33341.99>, R2 <0.17>, CV_score <0.02>